# Stonks

Python code to support trading experiences. (The following is not investment advice. Invest at your own risk)

In its current form (01/05/2021) this code scrapes options info and performs some minor calculations, filtering and sorting to support 3 (4?) primary option trading strategies.


## Credit Spreads

Credit spreads are a synthetic option position created by selling an out of the money (OTM) contract (to receive credit or "premium") and buying an even further OTM option to limit maximum potential losses, though also decreasing maximum potential gain. Credit spreads can be done with calls or puts. 

The maximum potential profit of a credit spread is the difference in the premium the near-priced option is sold at minus the price of the further-price option purchased (i.e. the premium collected at open). The maximum potential loss is difference in the strike prices * 100 - premium collected at open.

## The Wheel

The wheel is a cyclical combination of two option trading strategies (selling cash covered puts and selling covered calls). 
The Wheel works as follows:
1. The trader sells a cash-covered put against a given ticker at the chosen strike price (minimum cash required = 100 * strike price - premium contract is sold for).
2. If the contract expires worthless, the trader collects their premium and goes back to (1) where they again sell a cash-covered put. If the contract is in the money (ITM) at the time of expiration they are assigned and purchase 100 shares of the underlying assest at the strike price of their contract and proceeds to (3).
3. The trader sells a covered call (covered by the 100 shares they were assigned when their put expired ITM) at or above the strike price they were assigned the stock at (selling at a lower strike price risks loss of money in the case of assignmnet).
4. If the covered call expires worthless, the trader collects their premium and repeats (3) until assignmnet. If the covered call expires ITM, the 100 shares of the asset will be called away, for which they will recieve both the premium of the contract and the strike price * 100 for the shares sold. 

## Poor Man's Covered Calls (PMCC)

Poor man's covered calls is a strategy in which the trader synthetically creates a covered call position by purchasing a long dated call (typically ITM) as an alternative to holding 100 shares of the underlying asset, against which they sell shorted dated (synthetically covered) calls (OTM). Doing so raises the break-even strike price against which they can sell calls (i.e. smaller premiums) but are cheaper than purchasing 100 shares of the underlying asset (potentially increasing return per capital invested). 

The typical recommendation for the long dated (purchased) call is a delta in the range of 0.8-0.9. 
There are two standard recommendations for the short dated (sold call)

Option 1. A delta of 0.3

OR

Option 2. A strike price of at least the premium paid for the long dated call + the strike price of the long dated call (i.e. this represents the price at which the trader would essentially be paying for the underlying asset if they were to execute upon their contract).

A risk of the PMCC is that the underlying asset will increase in value to a value greater than the strike price of the short-dated call before its expiration date, thus putting this contract ITM. It is not advised for the trader to allow this contract to expire ITM, as it would require the execution of the long dated call to cover (and its time value will be greater than or equal to that of the short call). Instead, it is recommended for the trader to do one of two things.
1. Buy back (to close) the short call contract if the price is acceptable to the trader. In this situation the trader can simply reopen a new short dated call position at a higher strike price, thus keeping the PMCC position. 
2. Simultaneously buy back the short dated call contract (to close) and sell the long dated call contract (to close), thus closing the PMCC position. Doing so should (*should*) result in profit regardless of whether the short dated call was selected via the recommendation of Option 1 or Option 2. The increase in the price of the underlying asset resulted in an increase in the price of both the long and the short dated contracts, and as the long dated call had a higher delta the increase in its price should exceed the increase in the price of the short dated call. 


# The Code

The following code is written in Python 3.4. I have attempted to make it as easy as possible to be used by others, including those illiterate with Python or other programming languages. 

However, it will require some user input and changes in the inputs to reflect the trader's own personal goals, assessments, and risk tolerance. Each of these places will be marked clearly.

Not all functions are needed to run an individual strategy, it is a mix of tools to aid in the trading of credit spreads, the wheel, and PMCCs (though there are some overlaps in the blocks). 

Running this code does require the user to have an active account with Robinhood. If the user does not currently have a Robinhood account, I politely ask them to use my referal link when signing up (join.robinhood.com/conorf123). Doing so will grant each of us one free stock. That is better than 0 free stocks. 

Additionally, this code makes use of a number of Python libraries. Pressumably, if you were able to open this file in Jupyter Notebook then you are familiar in how to download Python libraries. If not, it is a simple as the following:
1. Open a terminal window (Command Prompt on Windows)
2. Type in the following: pip install [libray-name]

Example:

<code>pip install robin_stocks</code>

If that does not work for you, find a tutorial on Youtube.  

### Setup and libraries

This first cell does require each user of this document to make a few changes. 

This block handles signing into Robinhood so that we can make use of the API. I am not giving out my Robinhood username and password with the code, so you are going to have to fill in your own. 

If you are computer illiterate, do not worry about the top block of code, this is my way of loading in my password from a text file without actually having to have my actual password in plain text on a file. All you need to do is:
1. Find the function <code>RH_log_in_pleb()</code>
2. Find the first line of code in this function (i.e. <code>username=""</code>)
3. Update the right side of this equal sign to be the email address of your Robinhood account inside of double-quotes (e.g. <code>username="conorfoley@example.com"</code>
4. Find the second line of code in this function (i.e. <code>password=""</code>)
5. Update the right side of this equal sign to be the password of your Robinhood account inside of double-quotes (e.g. <code>password="Buy$TSLAcalls"</code>)
6. Hit run

In [2]:
def RH_log_in():
    import sys
    sys.path.append("C:/Users/Conor/Documents/Wheel/Misc")
    from caesar import cipher_decrypt

    f=open("C:/Users/Conor/Documents/Wheel/Misc/rhsignin.txt","r")
    lines=f.readlines()
    username=lines[0].rstrip()
    password=lines[1].rstrip()
    key = int(lines[2].rstrip())
    f.close()
    robin_stocks.authentication.login(username=username, password=cipher_decrypt(password, key), expiresIn=86400, scope='internal', by_sms=True, store_session=True)
    
def RH_log_in_pleb():
    username=""
    password=""
    robin_stocks.authentication.login(username=username, password=password, expiresIn=86400, scope='internal', by_sms=True, store_session=True)

This block of code handles loading the necessary libraries for the main code functions, and it also performs the log in to Robinhood. 

The only change necessary here is that if you made updates to the <code>RH_log_in_pleb()</code> function above then make sure that there **IS** a **#** in front of the line of code <code> RH_log_in()</code> and there **IS NOT** a **#** in front of the line of code <code>RH_log_in_pleb()</code>

Errors in this block will either be from
1. Issues not having all of the necessary libraries setup. Go install them with pip
2. Not entering your Robinhood information correctly above

In [3]:
import robin_stocks
import pandas as pd
import contextlib
import time
import math
from datetime import datetime
import subprocess
from func_timeout import func_timeout, FunctionTimedOut
from yahoo_fin import options
from yahoo_fin import stock_info as si

RH_log_in()
#RH_log_in_pleb()

### Deprecated code

The following blocks of code are considered deprecated. They were items I used during the evolution of this tool, but have since stopped using/stopped caring about. I include them in case some motivated, computer literate individual using this wants to try to work them back into their pipeline, but they are unnecessary for the cookie cutter version I am walking the average person through. 

In [ ]:
def useFinviz():
    proc = subprocess.Popen(['py', 'C:/Users/Conor/Documents/test.py'], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    myTickersUnprocessed = proc.communicate()[0]
    myList = myTickersUnprocessed.decode("utf-8").split('\r\n')
    myList.pop()
    return(myList)

In [ ]:
def updateCurrentPositions():
    global currentTickers
    global currentStrikes
    global currentExpDates
    currentTickers = []
    currentStrikes = []
    currentExpDates = []
    
    myCurrentPositions = robin_stocks.options.get_open_option_positions()

    for item in myCurrentPositions:
        tmpInfo = robin_stocks.get_option_instrument_data_by_id(item['option_id'])
        if item['type'] == 'short' and tmpInfo['type'] == 'put':
            #print(tmpInfo['chain_symbol'], '$' + str(round(float(tmpInfo['strike_price']),2)), item['type'], tmpInfo['type'], tmpInfo['expiration_date'])
            currentTickers.append(tmpInfo['chain_symbol'])
            currentStrikes.append(round(float(tmpInfo['strike_price']),2))
            currentExpDates.append(tmpInfo['expiration_date'])

In [ ]:
SP500andFinviz = useFinviz()
sp500results = si.tickers_sp500()
SP500andFinviz.extend(sp500results)

### My hard work

The following blocks of code are functions that do the scraping, computations, filtering, and sorting. These **DO NOT** need to be changed in anyway, but they do need to be run. 

Again, if you do not know what you are doing **DO NOT TOUCH THESE**. Simply run them and move along. 

In [4]:
def days_between(d1, d2):
    d1 = datetime.strptime(d1, "%Y-%m-%d")
    d2 = datetime.strptime(d2, "%Y-%m-%d")
    return (d2 - d1).days

def days_to_yearly_return(percentReturn, daysLeft):
    return( percentReturn * (52/(1+max(0,math.floor(daysLeft/7)))))

def to_float_with_none_check(n):
    if n is None:
        return 0
    else:
        return float(n)

In [5]:
def getInfoForSpecificContract(ticker, optionType, strikePrice, expirationDate):
    
    option_info = options.get_options_chain(ticker, date = expirationDate)   
    #option_info = robin_stocks.options.find_options_for_stock_by_expiration(symbol=ticker, expirationDate=expirationDate, optionType=optionType)
    
    return(option_info[optionType][option_info[optionType]['Strike']==strikePrice])

In [6]:
def buildExtendedProfile(partialProfileDF, premiumType, brokerFee):
    currentDate = datetime.today().strftime ('%Y-%m-%d')
    partialProfileDF['Days to Expiration'] = partialProfileDF.apply(lambda row : days_between(currentDate, row['Expiration']), axis = 1) 
    partialProfileDF['Capital Required'] = partialProfileDF.apply(lambda row : float(row['Strike'])*100, axis = 1) 
    partialProfileDF['Total Premium'] = partialProfileDF.apply(lambda row : float(row[premiumType])*100, axis = 1) 
    partialProfileDF['Total Premium'] = partialProfileDF['Total Premium'] - brokerFee
    partialProfileDF['Gross % to Gain'] = partialProfileDF['Total Premium'] / partialProfileDF['Capital Required']
    partialProfileDF['Yearly % Return'] = partialProfileDF.apply(lambda row : days_to_yearly_return(row['Gross % to Gain'], row['Days to Expiration']), axis = 1) 
    partialProfileDF['Yearly (Offset)'] = partialProfileDF.apply(lambda row : days_to_yearly_return(row['Gross % to Gain'], row['Days to Expiration'] - 7), axis = 1) 
    
    partialProfileDF['Strike'] = partialProfileDF.apply(lambda row : float(row['Strike']), axis = 1)
    #partialProfileDF['Probability'] = partialProfileDF.apply(lambda row : float(row['Probability']), axis = 1)
    partialProfileDF['Bid'] = partialProfileDF.apply(lambda row : float(row['Bid']), axis = 1)
    
    return(partialProfileDF)

In [7]:
def highlight_greaterthan(x):
    if x.Have == 1:
        return ['background-color: yellow']*x.shape[0]
    elif x.Have == 0.5:
        return ['background-color: #B2FFFF']*x.shape[0]
    else:
        return ['background-color: white']*x.shape[0]

In [8]:
def sellingPuts_AsLimitBuy(ticker, maxStrike=math.inf, OTMpercent=0, viewNum=20, latestDate='3000-12-31', brokerFee=0):
    #get all expiration dates available
    currentDate = datetime.today().strftime ('%Y-%m-%d')

    expirationDates = options.get_expiration_dates(ticker)
    expirationDates = [datetime.strptime(item, "%B %d, %Y").strftime("%Y-%m-%d") for item in expirationDates if days_between(datetime.strptime(item, "%B %d, %Y").strftime("%Y-%m-%d"), latestDate)>=0]

    puts = pd.DataFrame (columns = ['Strike','Ask','Bid', 'Price', 'Expiration'])

    for j in range(0, len(expirationDates)):

        try:
            with contextlib.redirect_stdout(None):
                option_info = options.get_puts(ticker, date = expirationDates[j])        
        except:
            print("EXCEPTION: ", expirationDates[j], " - ", ticker)
            continue

        tmpDF = option_info.reindex(columns=['Strike', 'Ask', 'Bid'])
        tmpDF['Expiration'] = expirationDates[j]
        puts = puts.append(tmpDF, sort = True)
       
    try:
        price = round(si.get_live_price(ticker),2)
    except:
        print("Issue getting price for", ticker)
        price = -1
        

    puts['Price'] = price
    
    puts = puts.replace('-', '0')
 
    if puts.isnull().values.any() == True:
        print("Null values in", ticker, "- skipping")
        return(pd.DataFrame())

    if puts.shape[0] > 0:
        puts = buildExtendedProfile(puts, 'Bid', brokerFee=brokerFee)
    else:
        return(pd.DataFrame())
    
    puts = puts.drop_duplicates(keep='first')
    
    hasBid = puts['Bid'] > 0
    goodStrikes = puts['Strike'] <= min(maxStrike, price * (1 - OTMpercent))

    goodPuts = puts[hasBid & goodStrikes]
    return(goodPuts.sort_values(by=['Yearly % Return'])[-viewNum:])
    


In [9]:
def sellingPuts_AsLimitBuy_SCANNER(tickerList, strikeList, OTMpercent=0, latestDate='3000-12-31', brokerFee = 0, offsetSort = 0):

    columnTitles = ['Ticker', 'Strike', 'Price', 'Ask', 'Bid','Expiration', 'Days to Expiration', 'Capital Required', 'Total Premium', 'Gross % to Gain', 'Yearly % Return', 'Yearly (Offset)']
    bestOptionsList = pd.DataFrame (columns = columnTitles)
    
    for i in range(0, len(tickerList)):
        #try:
        print(tickerList[i])
        if i + 1 > len(strikeList):
            tmpStats = sellingPuts_AsLimitBuy(ticker = tickerList[i], maxStrike = math.inf, OTMpercent = OTMpercent, latestDate = latestDate, viewNum = 5, brokerFee = brokerFee)
        else:
            tmpStats = sellingPuts_AsLimitBuy(ticker = tickerList[i], maxStrike = strikeList[i], OTMpercent = OTMpercent, latestDate = latestDate, viewNum = 5, brokerFee = brokerFee)
        if tmpStats.empty:
            continue
        tmpStats['Ticker']=tickerList[i]
        
        #bestOptionsList.append(tmpStats, ignore_index=True)
        bestOptionsList = bestOptionsList.append(tmpStats, ignore_index = True, sort = True)
    
    bestOptionsList = bestOptionsList.reindex(columns=columnTitles)
    if offsetSort == 0:
        bestOptionsList = bestOptionsList.sort_values(by=['Yearly % Return']) 
    else:
        bestOptionsList = bestOptionsList.sort_values(by=['Yearly (Offset)']) 
        
        
    return(bestOptionsList)

In [10]:
def styleScanner(df):
    df=df.reset_index(drop=True)
    i_have = pd.IndexSlice[df.loc[df['Have']==1].index, 'Yearly % Return']
    i_match = pd.IndexSlice[df.loc[df['Have']==0.5].index, 'Yearly % Return']
    i_dont = pd.IndexSlice[df.loc[df['Have']==0].index, 'Yearly % Return']

    columnTitles = ['Ticker', 'Price', 'Strike','Ask','Bid','Expiration', 'Days to Expiration', 'Capital Required', 'Total Premium', 'Gross % to Gain', 'Yearly % Return', 'Have', 'Yearly (Offset)']
    df2=df.reset_index(drop=True) \
        .reindex(columns=columnTitles) \
        .style.bar(subset=i_have, color=['yellow']) \
        .bar(subset=i_match, color=['#B2FFFF']) \
        .bar(subset=i_dont, align='mid', color=['#5fba7d']) \
        .apply(highlight_greaterthan, axis=1) \
        .hide_columns(['Have']) \
        .hide_index()
    return(df2)

In [11]:
def styleCalls(df):
    df=df.reset_index(drop=True)
    i_have = pd.IndexSlice[df.loc[df['Have']==1].index, 'Yearly Return']
    i_match = pd.IndexSlice[df.loc[df['Have']==0.5].index, 'Yearly Return']
    i_dont = pd.IndexSlice[df.loc[df['Have']==0].index, 'Yearly Return']

    columnTitles = ['Ticker', 'Strike', 'Price', 'Ask', 'Bid', 'Expiration', 'Days to Expiration', 'Total Premium', 'Total Return on Execution', 'Yearly Return', 'Yearly Return (Offset)',  'Have']
    df2=df.reset_index(drop=True) \
        .reindex(columns=columnTitles) \
        .style.bar(subset=i_have, color=['yellow']) \
        .bar(subset=i_match, color=['#B2FFFF']) \
        .bar(subset=i_dont, align='mid', color=['#5fba7d']) \
        .apply(highlight_greaterthan, axis=1) \
        .hide_columns(['Have']) \
        .hide_index()
    return(df2)

In [12]:
def stylePuts(df):
    df=df.reset_index(drop=True)
    
    columnTitles = ['Ticker', 'Price', 'Strike','Ask','Bid','Expiration', 'Days to Expiration', 'Capital Required', 'Total Premium', 'Gross % to Gain', 'Yearly % Return']
    df2=df.reset_index(drop=True) \
        .reindex(columns=columnTitles) \
        .style.bar(subset = ['Yearly % Return'], align='mid', color=['#5fba7d']) \
        .hide_index()
    return(df2)

In [13]:
def sellingPuts_AsLimitBuy_SCANNER_with_currentPositions(tickerWatchList, strikeWatchList, tickersHolding, strikesHolding, expirationsHolding, OTMpercent=0, latestDate='3000-12-31', brokerFee = 0, offsetSort = 0):
    
    #Get positions we are scanning
    scanResults = sellingPuts_AsLimitBuy_SCANNER(tickerWatchList, strikeWatchList, OTMpercent, latestDate=latestDate, brokerFee = brokerFee, offsetSort = offsetSort)
    
    scanResults['Have'] = 0
    
    columnTitles = ['Ticker', 'Strike', 'Price', 'Ask','Bid','Expiration', 'Days to Expiration', 'Capital Required', 'Total Premium', 'Gross % to Gain', 'Yearly % Return', 'Yearly (Offset)']
    currentPositionsList = pd.DataFrame (columns = columnTitles)
    currentPositionsListEntry = pd.DataFrame (columns = columnTitles)
    
    #Get current positions
    for i in range(0, len(tickersHolding)):
        
        tmp = getInfoForSpecificContract(tickersHolding[i], 'puts', strikesHolding[i], expirationsHolding[i])

        try:
            tmpDF = tmp.reindex(columns=['Strike', 'Ask', 'Bid'])
            tmpDF['Expiration'] = expirationsHolding[i]
        except:
            print("Issue getting: ", tickersHolding[i], strikesHolding[i], expirationsHolding[i])
            continue

        tmpDF['Ticker'] = tickersHolding[i]
        if tmpDF.empty:
            print("***** Issue with current position:", tickersHolding[i], strikesHolding[i], expirationsHolding[i])
            continue
        try:
            price = round(si.get_live_price(tickersHolding[i]),2)
        except:
            print("Issue getting price for", tickersHolding[i])
            price = -1

        tmpDF['Price'] = price

        currentPositionsList = currentPositionsList.append(buildExtendedProfile(tmpDF, 'Ask', -brokerFee)) #negative broker fee because we are buying to close
        currentPositionsListEntry = currentPositionsListEntry.append(buildExtendedProfile(tmpDF, 'Bid', brokerFee))
        
    currentPositionsList['Have'] = 1
    currentPositionsListEntry['Have'] = 0.5
    
    columnTitles = ['Ticker', 'Strike', 'Price', 'Ask','Bid','Expiration', 'Days to Expiration', 'Capital Required', 'Total Premium', 'Gross % to Gain', 'Yearly % Return', 'Have', 'Yearly (Offset)']
    
    if offsetSort == 0:
        outTable = scanResults.append(currentPositionsList).append(currentPositionsListEntry).sort_values(by=['Have']).drop_duplicates(subset=['Ticker', 'Strike', 'Price', 'Ask','Bid','Expiration', 'Days to Expiration', 'Capital Required', 'Total Premium', 'Gross % to Gain', 'Yearly % Return', 'Yearly (Offset)'], keep = 'last').sort_values(by=['Yearly % Return'])
    else:
        outTable = scanResults.append(currentPositionsList).append(currentPositionsListEntry).sort_values(by=['Have']).drop_duplicates(subset=['Ticker', 'Strike', 'Price', 'Ask','Bid','Expiration', 'Days to Expiration', 'Capital Required', 'Total Premium', 'Gross % to Gain', 'Yearly % Return', 'Yearly (Offset)'], keep = 'last').sort_values(by=['Yearly (Offset)'])

    #outTable.reset_index(drop=True).reindex(columns=columnTitles).style.bar(subset=['Yearly % Return'], align='mid', color=['#5fba7d']).apply(highlight_greaterthan, axis=1)

    
    return(outTable)

In [14]:
def sellingCalls_AsLimitSell(ticker, minStrike=0, OTMpercent = 0, viewNum=20, latestDate='3000-12-31', sort_by_theta = 0, brokerFee=0):
    #get all expiration dates available
    expirationDates = options.get_expiration_dates(ticker)
    expirationDates = [datetime.strptime(item, "%B %d, %Y").strftime("%Y-%m-%d") for item in expirationDates if days_between(datetime.strptime(item, "%B %d, %Y").strftime("%Y-%m-%d"), latestDate)>=0]

    currentDate = datetime.today().strftime ('%Y-%m-%d')

    expirationDates = options.get_expiration_dates(ticker)
    expirationDates = [datetime.strptime(item, "%B %d, %Y").strftime("%Y-%m-%d") for item in expirationDates if days_between(datetime.strptime(item, "%B %d, %Y").strftime("%Y-%m-%d"), latestDate)>=0]

    calls = pd.DataFrame (columns = ['Strike','Ask','Bid','Delta', 'Theta', 'Price', 'Expiration'])

    for j in range(0, len(expirationDates)):

        try:
            with contextlib.redirect_stdout(None):
                #option_info = options.get_calls(ticker, date = expirationDates[j])    

                pre_option_info = robin_stocks.find_options_for_stock_by_expiration(ticker, expirationDate=expirationDates[j], optionType='call')
            data = [ (item['strike_price'], item['ask_price'], item['bid_price'],  to_float_with_none_check(item['delta']), to_float_with_none_check(item['theta'])) for item in pre_option_info]
            option_info  = pd.DataFrame(data, columns = ['Strike', 'Ask', 'Bid', 'Delta', 'Theta'])
        except:
            print("EXCEPTION: ", expirationDates[j], " - ", ticker)
            continue

        tmpDF = option_info.reindex(columns=['Strike', 'Ask', 'Bid', 'Delta', 'Theta'])
        tmpDF['Expiration'] = expirationDates[j]
        calls = calls.append(tmpDF, sort = True)

    try:
        price = round(si.get_live_price(ticker),2)
    except:
        print("Issue getting price for", ticker)
        price = -1


    calls['Price'] = price

    calls = calls.replace('-', '0')

    if calls.isnull().values.any() == True:
        print("Null values in", ticker, "- skipping")
        return(pd.DataFrame())

    if calls.shape[0] > 0:
        calls = buildExtendedCallProfile(calls, 'Bid', brokerFee=brokerFee, ticker = ticker)
    else:
        return(pd.DataFrame())

    calls = calls.drop_duplicates(keep='first')

    hasBid = calls['Bid'] > 0
    goodStrikes = calls['Strike'] >= max(minStrike, price * (1 + OTMpercent))

    goodCalls = calls[hasBid & goodStrikes]
    if sort_by_theta == 0:
        return(goodCalls.sort_values(by=['Yearly Return'])[-viewNum:])
    else: 
        return(goodCalls.sort_values(by=['Theta'], ascending=False)[-viewNum:])
    

    


In [15]:
def buildExtendedCallProfile(df, premiumType, brokerFee, ticker):
    currentDate = datetime.today().strftime ('%Y-%m-%d')
    currentPrice = round(si.get_live_price(ticker),2)
    df['Days to Expiration'] = df.apply(lambda row : days_between(currentDate, row['Expiration']), axis = 1) 
    df['Price'] = currentPrice
    df['Total Premium'] = df.apply(lambda row : float(row[premiumType])*100, axis = 1) 
    df['Total Premium'] = df['Total Premium'] - brokerFee
    df['Yearly Return'] = df.apply(lambda row : round(days_to_yearly_return(row['Total Premium'], row['Days to Expiration']), 2), axis = 1)
    df['Yearly Return (Offset)'] = df.apply(lambda row : round(days_to_yearly_return(row['Total Premium'], row['Days to Expiration'] - 7), 2), axis = 1) 
    df['Strike'] = df.apply(lambda row : float(row['Strike']), axis = 1)
    df['Bid'] = df.apply(lambda row : float(row['Bid']), axis = 1)
    df['priceDiff'] = df.apply(lambda row : abs(round(currentPrice - row['Strike'], 2)) , axis = 1)
    
    #df.loc[df['Strike'] < currentPrice, 'Note'] = 'ITM by $'
    #df.loc[df['Strike'] > currentPrice, 'Note'] = 'OTM by $'
    
    #df['Total Return on Execution'] = df.apply(lambda row : row['Strike']*100 + row['Total Premium'], axis= 1)
    #df.loc[df['Strike'] < currentPrice, 'Yearly Return'] = df.apply(lambda row : row['Total Return on Execution'], axis = 1)
    
    #df['Note'] = df.apply(lambda row : str(row['Note'] + str(row['priceDiff'])), axis=1)
    #df.loc[df['Strike'] == currentPrice, 'Note'] = 'ATM'
    
    df = df.drop(['priceDiff'], axis = 1)
    
    return(df)

In [16]:
def sellingCalls_AsLimitSell_with_currentPositions(ticker, strikesHolding, expirationsHolding, minStrike=0, OTMpercent=0, latestDate='3000-12-31', brokerFee = 0):
    
    scanResults = sellingCalls_AsLimitSell(ticker=ticker, minStrike=minStrike, OTMpercent = OTMpercent, viewNum=20, latestDate=latestDate, brokerFee=brokerFee)
    
    scanResults['Have'] = 0
    scanResults['Ticker'] = ticker
    
    columnTitles = ['Ticker', 'Strike', 'Price', 'Ask','Bid','Expiration', 'Days to Expiration', 'Total Premium', 'Yearly Return']
    currentPositionsList = pd.DataFrame (columns = columnTitles)
    currentPositionsListEntry = pd.DataFrame (columns = columnTitles)
    
    #Get current positions
    for i in range(0, len(strikesHolding)):
        
        tmp = getInfoForSpecificContract(ticker, 'calls', strikesHolding[i], expirationsHolding[i])

        try:
            tmpDF = tmp.reindex(columns=['Strike', 'Ask', 'Bid'])
            tmpDF['Expiration'] = expirationsHolding[i]
        except:
            print("Issue getting: ", ticker, strikesHolding[i], expirationsHolding[i])
            continue

        tmpDF['Ticker'] = ticker
        try:
            price = round(si.get_live_price(ticker),2)
        except:
            print("Issue getting price for", ticker)
            price = -1


        currentPositionsList = currentPositionsList.append(buildExtendedCallProfile(tmpDF, 'Ask', -brokerFee, ticker = ticker)) #negative broker fee because we are buying to close
        currentPositionsListEntry = currentPositionsListEntry.append(buildExtendedCallProfile(tmpDF, 'Bid', brokerFee, ticker = ticker))
        
    currentPositionsList['Have'] = 1
    currentPositionsListEntry['Have'] = 0.5
    

    outTable = scanResults.append(currentPositionsList).append(currentPositionsListEntry).sort_values(by=['Have']).drop_duplicates(subset=['Ticker', 'Strike','Ask','Bid','Expiration', 'Days to Expiration', 'Total Premium', 'Yearly Return'], keep = 'last').sort_values(by=['Yearly Return'])
    
    return(outTable)
    
    
    
    

In [17]:
def callCreditSpreadForTicker(ticker, minStrike=0, OTMpercent=0, brokerFee=0):
    currentDate = datetime.today().strftime ('%Y-%m-%d')
    currentPrice = round(si.get_live_price(ticker),2)
    expirationDates = options.get_expiration_dates(ticker)

    calls = pd.DataFrame (columns = ['Strike','Ask','Bid', 'Expiration'])

    option_info = options.get_calls(ticker, expirationDates[0]) 

    tmpDF = option_info.reindex(columns=['Strike', 'Ask', 'Bid'])
    tmpDF['Expiration'] = expirationDates[0]

    calls = calls.append(tmpDF, sort = True)

    calls = calls.drop_duplicates(keep='first')

    hasBid = calls['Bid'] > 0
    goodStrikes = calls['Strike'] >= max(minStrike, currentPrice * (1 + OTMpercent))

    goodCalls = calls[hasBid & goodStrikes]

    goodCalls = goodCalls.sort_values(by=['Strike']) 

    goodCalls['Current Price'] = currentPrice
    goodCalls['Lower Strike'] = goodCalls['Strike']
    goodCalls['Cover Strike'] = goodCalls['Strike'].shift(-1)
    goodCalls['Cost to Cover'] = goodCalls['Ask'].shift(-1)
    goodCalls['Credit for Sale'] = goodCalls['Bid']
    goodCalls['Total Credit'] = goodCalls['Credit for Sale'] - goodCalls['Cost to Cover']
    goodCalls['Collateral/Max Risk'] = 100 * (goodCalls['Cover Strike'] - goodCalls['Lower Strike'])
    goodCalls['% Return'] = 100 * (goodCalls['Total Credit'] / goodCalls['Collateral/Max Risk'])
    goodCalls['% OTM'] = (goodCalls['Lower Strike'] - goodCalls['Current Price']) / goodCalls['Current Price']

    creditToMake = goodCalls['Total Credit'] > 0

    columnTitles = ['Expiration', 'Current Price', 'Lower Strike', 'Cover Strike', 'Credit for Sale', 'Cost to Cover', 'Total Credit', 'Collateral/Max Risk', '% Return', '% OTM']
    
    return(goodCalls[creditToMake ].reindex(columns=columnTitles))

In [18]:
def putCreditSpreadForTicker(ticker, maxStrike=math.inf, OTMpercent=0, brokerFee=0):
    
    
    currentDate = datetime.today().strftime ('%Y-%m-%d')
    currentPrice = round(si.get_live_price(ticker),2)
    expirationDates = options.get_expiration_dates(ticker)

    puts = pd.DataFrame (columns = ['Strike','Ask','Bid', 'Expiration'])

    option_info = options.get_puts(ticker, expirationDates[0]) 

    tmpDF = option_info.reindex(columns=['Strike', 'Ask', 'Bid'])
    tmpDF['Expiration'] = expirationDates[0]

    puts = puts.append(tmpDF, sort = True)

    puts = puts.drop_duplicates(keep='first')

    hasBid = puts['Bid'] > 0
    goodStrikes = puts['Strike'] <= min(maxStrike, currentPrice * (1 - OTMpercent)) 

    goodPuts = puts[hasBid & goodStrikes]
    goodPuts = goodPuts.sort_values(by=['Strike']) 

    goodPuts['Current Price'] = currentPrice
    goodPuts['Higher Strike'] = goodPuts['Strike']
    goodPuts['Cover Strike'] = goodPuts['Strike'].shift()
    goodPuts['Cost to Cover'] = goodPuts['Ask'].shift()
    goodPuts['Credit for Sale'] = goodPuts['Bid']
    goodPuts['Total Credit'] = goodPuts['Credit for Sale'] - goodPuts['Cost to Cover']
    goodPuts['Collateral/Max Risk'] = 100 * (goodPuts['Higher Strike'] - goodPuts['Cover Strike'])
    goodPuts['% Return'] = 100 * (goodPuts['Total Credit'] / goodPuts['Collateral/Max Risk'])
    goodPuts['% OTM'] = (goodPuts['Current Price'] - goodPuts['Higher Strike']) / goodPuts['Current Price']
    

    creditToMake = goodPuts['Total Credit'] > 0

    columnTitles = ['Expiration', 'Current Price', 'Higher Strike', 'Cover Strike', 'Credit for Sale', 'Cost to Cover', 'Total Credit', 'Collateral/Max Risk', '% Return', '% OTM']
    return(goodPuts[creditToMake].reindex(columns=columnTitles))

In [19]:
def callCreditSpread_SCANNER(tickerList, sortBy = '% OTM', maxCollateral = math.inf, OTMpercent = 0, brokerFee = 0):
    columnTitles = ['Ticker', 'Expiration', 'Current Price', 'Lower Strike', 'Cover Strike', 'Credit for Sale', 'Cost to Cover', 'Total Credit', 'Collateral/Max Risk', '% Return', '% OTM']
    bestOptionsList = pd.DataFrame (columns = columnTitles)


    for i in range(0, len(tickerList)):
        print(tickerList[i])
        try:
            tmpDF = callCreditSpreadForTicker(tickerList[i], 0, OTMpercent = OTMpercent, brokerFee = brokerFee)
        except:
            print("Issue with ", tickerList[i])
            continue
        tmpDF['Ticker'] = tickerList[i]

        bestOptionsList = bestOptionsList.append(tmpDF, ignore_index = True, sort = True)

    bestOptionsList = bestOptionsList.reindex(columns=columnTitles)
    
    collateralFilter = bestOptionsList['Collateral/Max Risk'] <= maxCollateral
    
    return(bestOptionsList[collateralFilter].sort_values(by = sortBy))

In [20]:
def putCreditSpread_SCANNER(tickerList, sortBy = '% OTM', maxCollateral = math.inf, OTMpercent = 0, brokerFee = 0):
    columnTitles = ['Ticker', 'Expiration', 'Current Price', 'Higher Strike', 'Cover Strike',  'Credit for Sale', 'Cost to Cover', 'Total Credit', 'Collateral/Max Risk', '% Return', '% OTM']
    bestOptionsList = pd.DataFrame (columns = columnTitles)


    for i in range(0, len(tickerList)):
        print(tickerList[i])
        try:
            tmpDF = putCreditSpreadForTicker(tickerList[i], math.inf, OTMpercent = OTMpercent, brokerFee = brokerFee)
        except:
            print("Issue with ", tickerList[i])
            continue
        tmpDF['Ticker'] = tickerList[i]

        bestOptionsList = bestOptionsList.append(tmpDF, ignore_index = True, sort = True)

    bestOptionsList = bestOptionsList.reindex(columns=columnTitles)
    
    collateralFilter = bestOptionsList['Collateral/Max Risk'] <= maxCollateral
    
    return(bestOptionsList[collateralFilter].sort_values(by = sortBy))

In [21]:
sp500results = si.tickers_sp500() #get a list of all tickers in the S&P 500 into a list

In [22]:
def find_LEAPS_by_break_even_debit(ticker, minDate='2021-01-01', maxDelta=0.9):
    with contextlib.redirect_stdout(None):
        expirationDates = list(set(robin_stocks.options.find_tradable_options_for_stock(ticker, optionType = 'call', info = 'expiration_date')))

    calls = pd.DataFrame (columns = ['Ticker', 'Strike','Ask','Bid', 'Price', 'Expiration'])

    for j in range(0, len(expirationDates)):
        try:
            with contextlib.redirect_stdout(None):
                pre_option_info = robin_stocks.find_options_for_stock_by_expiration(ticker, expirationDate=expirationDates[j], optionType='call')
            data = [ (item['strike_price'], item['ask_price'], item['bid_price'],  to_float_with_none_check(item['delta'])) for item in pre_option_info]
            option_info  = pd.DataFrame(data, columns = ['Strike', 'Ask', 'Bid', 'Delta'])
        except:
            print(expirationDates[j], "issue")
            continue
        tmpDF = option_info.reindex(columns=['Strike', 'Ask', 'Bid', 'Delta'])
        tmpDF['Expiration'] = expirationDates[j]

        calls = calls.append(tmpDF, sort = True)
    try:
        price = round(si.get_live_price(ticker),2)
    except:
        print("Issue getting price for", ticker)
        price = -1    
    calls['Price'] = price
    calls['Ticker'] = ticker
    calls = calls.replace('-', '0')

    currentDate = datetime.today().strftime ('%Y-%m-%d')
    #calls['Delta'] = calls.apply(lambda row :  float(row['Delta']), axis= 1)
    calls['Break Even Debit'] = calls.apply(lambda row :  float(row['Ask']) + float(row['Strike']), axis= 1)
    calls['Days to Exp'] = calls.apply(lambda row : days_between(currentDate, row['Expiration']), axis = 1)
    calls['Cost per Day'] = calls.apply(lambda row: float(row['Ask']) * 100 / float(row['Days to Exp']), axis = 1)
    calls = calls[calls['Delta'] > 0]
    sortedCalls = calls[calls['Expiration'] >= minDate].sort_values(by='Break Even Debit')
    sortedCalls = sortedCalls[sortedCalls['Delta'] <= maxDelta]
    return(sortedCalls[:50])

In [23]:
def search_calls_by_delta(ticker, low_delta, high_delta, near_date = '2020-12-31', far_date = '3000-12-31'):
    with contextlib.redirect_stdout(None):
        expirationDates = list(set(robin_stocks.options.find_tradable_options_for_stock(ticker, optionType = 'call', info = 'expiration_date')))

    calls = pd.DataFrame (columns = ['Strike','Ask','Bid','Delta', 'Theta', 'Price', 'Expiration'])

    for j in range(0, len(expirationDates)):

        try:
            with contextlib.redirect_stdout(None):
                #option_info = options.get_calls(ticker, date = expirationDates[j])    

                pre_option_info = robin_stocks.find_options_for_stock_by_expiration(ticker, expirationDate=expirationDates[j], optionType='call')
            data = [ (item['strike_price'], item['ask_price'], item['bid_price'],  to_float_with_none_check(item['delta']), to_float_with_none_check(item['theta'])) for item in pre_option_info]
            option_info  = pd.DataFrame(data, columns = ['Strike', 'Ask', 'Bid', 'Delta', 'Theta'])
        except:
            print("EXCEPTION: ", expirationDates[j], " - ", ticker)
            continue

        tmpDF = option_info.reindex(columns=['Strike', 'Ask', 'Bid', 'Delta', 'Theta'])
        tmpDF['Expiration'] = expirationDates[j]
        calls = calls.append(tmpDF, sort = True)
    try:
        price = round(si.get_live_price(ticker),2)
    except:
        print("Issue getting price for", ticker)
        price = -1    
    calls['Price'] = price
    calls['Ticker'] = ticker
    calls = calls.replace('-', '0')

    currentDate = datetime.today().strftime ('%Y-%m-%d')
    #calls['Delta'] = calls.apply(lambda row :  float(row['Delta']), axis= 1)
    calls['Break Even Debit'] = calls.apply(lambda row :  float(row['Ask']) + float(row['Strike']), axis= 1)
    calls['Days to Exp'] = calls.apply(lambda row : days_between(currentDate, row['Expiration']), axis = 1)
    calls['Cost per Day'] = calls.apply(lambda row: float(row['Ask']) * 100 / float(row['Days to Exp']), axis = 1)
    calls = calls[calls['Delta'] > 0]

    lowDates = calls['Expiration'] >= near_date
    highDates = calls['Expiration'] <= far_date

    calls = calls[lowDates & highDates]
    sortedCalls = calls.sort_values(by='Delta', ascending=False)

    lowDeltas = sortedCalls['Delta'] > low_delta
    highDeltas = sortedCalls['Delta'] <= high_delta

    goodCalls = sortedCalls[lowDeltas & highDeltas]
    return(goodCalls[:50])
    #sortedCalls = calls[calls['Expiration'] >= '2022-01-01'].sort_values(by='Break Even Debit')
    #sortedCalls[sortedCalls['Delta'] <= 0.9][:50]

### The sum of all of my efforts

These are the code blocks you need to care about, and whose outputs you use to aid in your trading decisions. 



## Credit Spreads

There are 4 different credit spread code blocks that can be run. All blocks look only for spreads at the next expiration date for that ticker. These code blocks are independent and do not require the running of all of them.  

1. <code>putCreditSpreadForTicker</code> search for a put credit spread for a specific ticker by either max strike (of the near money put) or the % out of the money of the near money put. 

2. <code>putCreditSpread_SCANNER</code>  scan over a list of tickers for a put credit spread by % out of the money of the near money put. Can also specify maximum collateral (e.g. a \\$1 spread between strike prices requires \\$100 collateral; a spread is not useful to the trader if they do not have the collateral required to hold it)

3. <code>callCreditSpreadForTicker</code> search for a call credit spread for a specific ticker by either min strike (of the near money call) or the % out of the money of the near money call. 

4. <code>callCreditSpread_SCANNER</code>  scan over a list of tickers for a call credit spread by % out of the money of the near money call. Can also specify maximum collateral (e.g. a \\$1 spread between strike prices requires \\$100 collateral; a spread is not useful to the trader if they do not have the collateral required to hold it)


In [ ]:
putCreditSpreadForTicker('TSLA', maxStrike=math.inf, OTMpercent=0.1, brokerFee=0)

In [ ]:
putCreditSpread_SCANNER(sp500results,                   maxCollateral = 500, OTMpercent = 0.1)

In [ ]:
callCreditSpreadForTicker('TSLA', minStrike=0, OTMpercent=0.1, brokerFee=0)

In [ ]:
callCreditSpread_SCANNER(sp500results,                   maxCollateral = 500, OTMpercent = 0.1)

## The Wheel

Remember that there are two parts to the wheel strategy: selling cash covered puts and then (if assigned) selling covered calls on the assigned stock. 

#### Selling Cash Covered Puts
The first block of code lets us search for good entry points for cash covered puts. We can either search by a single ticker in <code>sellingPuts_AsLimitBuy()</code> or over a group of them using <code>sellingPuts_AsLimitBuy_SCANNER</code>.
You can either specify the maximum strike price you want for a ticker, or the % out of the money that a strike price must be (if both specified then uses more stringent of the 2). Code defaults to only look at puts that are OTM at run time. 

Both functions have an input variable <code>latestDate</code>. Please ensure that this is a future date in the format of 'YYYY-MM-DD'. 

Results are sorted by Yearly % Return (roughly = return/collateral * 365/days to expiration) with the highest return contracts at the bottom of the list.

In [ ]:
sellingPuts_AsLimitBuy(ticker='TSLA', 
                       maxStrike=math.inf, 
                       OTMpercent=0, 
                       viewNum=20, 
                       latestDate='3000-12-31', 
                       brokerFee=0)

In [24]:
#BFandFinviz = useFinviz()
bottomFeeder =[
    'AAPL',
    'ABBV',
    'AMD',
    'AMZN',
    'ATVI',
    'BABA',
    'BAC',
    'BP',
    'BUD',
    'CRSP',
    'DAL',
    'DIS',
    'FB',
    'GOOG',
    'GOOGL',
    'HD',
    'HLT',
    'JNJ',
    'KO',
    'LOW',
    'MA',
    'MAR',
    'MGM',
    'MSFT',
    'NFLX',
    'NVDA',
    'PFE',
    'PYPL',
    'QQQ',
    'SHOP',
    'SNAP',
    'SPHD',
    'SPY',
    'SPYD',
    'T',
    'TGT',
    'TSLA',
    'UAL',
    'UBER',
    'V',
    'VZ',
    'WMT' 
]

#BFandFinviz.extend(bottomFeeder)


outTable=sellingPuts_AsLimitBuy_SCANNER(bottomFeeder, 
                               [], 
                               OTMpercent=0.015, 
                               latestDate='2021-02-28', 
                               brokerFee = 0, 
                               offsetSort = 0)
outTable['Have'] = 0
styleScanner(outTable)



AAPL
EXCEPTION:  2021-02-26  -  AAPL
ABBV
EXCEPTION:  2021-02-26  -  ABBV
AMD
EXCEPTION:  2021-02-26  -  AMD
AMZN
EXCEPTION:  2021-02-19  -  AMZN
EXCEPTION:  2021-02-26  -  AMZN
ATVI
EXCEPTION:  2021-02-26  -  ATVI
BABA
EXCEPTION:  2021-02-19  -  BABA
EXCEPTION:  2021-02-26  -  BABA
BAC
EXCEPTION:  2021-02-26  -  BAC
BP
EXCEPTION:  2021-02-19  -  BP
EXCEPTION:  2021-02-26  -  BP
BUD
EXCEPTION:  2021-02-19  -  BUD
EXCEPTION:  2021-02-26  -  BUD
CRSP
EXCEPTION:  2021-02-19  -  CRSP
EXCEPTION:  2021-02-26  -  CRSP
DAL
EXCEPTION:  2021-02-19  -  DAL
EXCEPTION:  2021-02-26  -  DAL
DIS
EXCEPTION:  2021-02-19  -  DIS
EXCEPTION:  2021-02-26  -  DIS
FB
EXCEPTION:  2021-02-26  -  FB
GOOG
EXCEPTION:  2021-02-19  -  GOOG
EXCEPTION:  2021-02-26  -  GOOG
GOOGL
EXCEPTION:  2021-02-19  -  GOOGL
HD
HLT
JNJ
KO
LOW
MA
MAR
MGM
MSFT
NFLX
NVDA
PFE
PYPL
QQQ
SHOP
SNAP
SPHD
SPY
SPYD
T
TGT
TSLA
UAL
UBER
V
VZ
WMT


Ticker,Price,Strike,Ask,Bid,Expiration,Days to Expiration,Capital Required,Total Premium,Gross % to Gain,Yearly % Return,Yearly (Offset)
JNJ,163.68,149,0.04,0.03,2021-02-26,7,14900,3,0.000201342,0.0052349,0.0104698
PFE,34.5,32,0.02,0.01,2021-02-26,7,3200,1,0.0003125,0.008125,0.01625
JNJ,163.68,152.5,0.1,0.05,2021-02-26,7,15250,5,0.000327869,0.00852459,0.0170492
PFE,34.5,33,0.03,0.02,2021-02-26,7,3300,2,0.000606061,0.0157576,0.0315152
PFE,34.5,32.5,0.03,0.02,2021-02-26,7,3250,2,0.000615385,0.016,0.032
KO,50.12,47.5,0.06,0.03,2021-02-26,7,4750,3,0.000631579,0.0164211,0.0328421
KO,50.12,47,0.06,0.03,2021-02-26,7,4700,3,0.000638298,0.0165957,0.0331915
MAR,136.98,120,0.55,0.1,2021-02-26,7,12000,10,0.000833333,0.0216667,0.0433333
JNJ,163.68,155,0.15,0.13,2021-02-26,7,15500,13,0.00083871,0.0218065,0.0436129
MAR,136.98,126,0.74,0.11,2021-02-26,7,12600,11,0.000873016,0.0226984,0.0453968


In [ ]:
x = sellingPuts_AsLimitBuy_SCANNER(sp500results, 
                               [], 
                               OTMpercent=0.025, 
                               latestDate='2021-02-28', brokerFee = 0, offsetSort = 0)

x['Have'] = 0

styleScanner(x)


#### Selling Covered Calls

Upon assignment of a cash covered put, you now move to phase 2 of the Wheel. 

In phase 2, the trader sells covered calls of the stock that they have been assigned. 

The <code>sellingCalls_AsLimitSell()</code> allows the trader to search for calls to sell for a given ticker at a minimum strike price, with expiration by a specified date. results can either be sorted by Yearly Return (roughly = premium * 365 / days to expiration) or by theta (time decay of the sold option). Default is yearly return, can change to theta by setting <code>sort_by_theta=1</code> in the function call.

In [ ]:
sellingCalls_AsLimitSell(ticker="TSLA", minStrike=750, OTMpercent = 0, viewNum=20, latestDate='2021-02-28', sort_by_theta = 0, brokerFee=0)

## Poor Man's Covered Calls

The PMCC is a way to synthetically create a covered call position by purchasing (ITM) LEAPS as a means to cover short dated short calls. Buying LEAPS raises your break even price on the stonk on the long side, but is cheaper to obtain than 100 shares of the underlying, essentially allowing the trader to increase the leverage of their collateral 

For example: with \\$10,000 a trader could purchase 100 shares of the underlying at \\$100 per share and sell covered calls at the \\$100 strike or they could buy two \\$65 strike contracts for \\$5000 each and sell two covered calls at the \\$115 strike (i.e. \\$65 strike + \\$50 per contract = \\$115 strike).

#### Long LEAPS

This part is as much art as it is science. There are a number of factors to consider that will all affect the trader's returns. Shorter dated, higher strike price contracts will cost the trader less money to enter, but give the trader less time to earn returns via selling shorts (and are more quickly suspect to theta/time decay). The trader must also take into account the delta of their long contract; if the price of the underlying shoots up past the strike price of your short contract it might be necessary to close your long position to cover. The higher the delta on the long, the greater chance that the trader does not lose money in these situations. 

Two different functions have been provided for assistance in finding LEAPS to long. <code>find_LEAPS_by_break_even_debit()</code> and <code>search_calls_by_delta()</code>

1. <code>find_LEAPS_by_break_even_debit()</code> filters calls by minimum expiration date (such that it can be sufficiently long dated for the trader's needs) and maximum delta (above 0.9 will generally return strikes near \\$0 in which case the trader is essentially just buying the underlying). Calls are sorted (lowest to high) of their break even debit (i.e strike price + price per contract), limiting results to only the top 50. 

For the short call, the trader can make use of the <code>sellingCalls_AsLimitSell()</code> originally seen in the Wheel section, using their break even debit as the <code>minStrike</code>. Note, that the lowest break even debit will get the highest premiums from selling calls at that price, but might not be the best in terms of return on investment over time. A more scientific approach to this question is currently a TODO, but at the moment picking a contract for this is as much art as it is science. 

2. <code>search_calls_by_delta()</code> filters calls by a given delta range (between <code>low_delta</code> and <code>high_delta</code>) and date range. Classical theory suggests that 0.8 is the appropriate delta for the long call in a PMCC and 0.3 for the short call. 



#### 1. <code>find_LEAPS_by_break_even_debit()</code>

In [ ]:
find_LEAPS_by_break_even_debit(ticker='NVDA', minDate='2022-06-01', maxDelta=0.9)

In [ ]:
sellingCalls_AsLimitSell(ticker="CRSP", minStrike=185, OTMpercent = 0, viewNum=20, latestDate='2021-02-28', sort_by_theta = 0, brokerFee=0)

#### 2. <code>search_calls_by_delta()</code>

In [25]:
search_calls_by_delta(ticker='NFLX', low_delta=0.75, high_delta=0.85, near_date = '2022-06-01', far_date = '3000-12-31')

,Ask,Bid,Delta,Expiration,Price,Strike,Theta,Ticker,Break Even Debit,Days to Exp,Cost per Day
62,229.300000,225.100000,0.846896,2023-01-20,540.63,350.0000,-0.056495,NFLX,579.30,700,32.757143
48,201.900000,199.550000,0.842141,2022-06-17,540.63,370.0000,-0.069783,NFLX,571.90,483,41.801242
48,222.450000,219.500000,0.835475,2023-01-20,540.63,360.0000,-0.059354,NFLX,582.45,700,31.778571
64,194.800000,192.200000,0.829896,2022-06-17,540.63,380.0000,-0.072741,NFLX,574.80,483,40.331263
34,216.050000,211.450000,0.824921,2023-01-20,540.63,370.0000,-0.061116,NFLX,586.05,700,30.864286
59,187.850000,185.400000,0.816945,2022-06-17,540.63,390.0000,-0.075889,NFLX,577.85,483,38.892340
3,209.550000,206.500000,0.812816,2023-01-20,540.63,380.0000,-0.064048,NFLX,589.55,700,29.935714
47,181.200000,178.700000,0.803630,2022-06-17,540.63,400.0000,-0.078948,NFLX,581.20,483,37.515528
22,203.250000,200.050000,0.801383,2023-01-20,540.63,390.0000,-0.066108,NFLX,593.25,700,29.035714
65,174.550000,171.850000,0.790285,2022-06-17,540.63,410.0000,-0.081570,NFLX,584.55,483,36.138716


In [ ]:
search_calls_by_delta(ticker='AMZN', low_delta=0.25, high_delta=0.35, near_date = '2021-01-01', far_date = '2021-02-28')

## Misc Functions

Code I have written but do not exactly know what to do with yet

In [ ]:
def calls_delta_per_collateral(ticker, minDate = '2020-12-31', minDelta = 0, reverse = 0):
    with contextlib.redirect_stdout(None):
        expirationDates = list(set(robin_stocks.options.find_tradable_options_for_stock(ticker, optionType = 'call', info = 'expiration_date')))

    calls = pd.DataFrame (columns = ['Strike','Ask','Bid','Delta', 'Theta', 'Price', 'Expiration'])

    for j in range(0, len(expirationDates)):

        try:
            with contextlib.redirect_stdout(None):
                #option_info = options.get_calls(ticker, date = expirationDates[j])    

                pre_option_info = robin_stocks.find_options_for_stock_by_expiration(ticker, expirationDate=expirationDates[j], optionType='call')
            data = [ (item['strike_price'], item['ask_price'], item['bid_price'],  to_float_with_none_check(item['delta']), to_float_with_none_check(item['theta'])) for item in pre_option_info]
            option_info  = pd.DataFrame(data, columns = ['Strike', 'Ask', 'Bid', 'Delta', 'Theta'])
        except:
            print("EXCEPTION: ", expirationDates[j], " - ", ticker)
            continue

        tmpDF = option_info.reindex(columns=['Strike', 'Ask', 'Bid', 'Delta', 'Theta'])
        tmpDF['Expiration'] = expirationDates[j]
        calls = calls.append(tmpDF, sort = True)
    try:
        price = round(si.get_live_price(ticker),2)
    except:
        print("Issue getting price for", ticker)
        price = -1    
    calls['Price'] = price
    calls['Ticker'] = ticker
    calls['Delta per Collateral'] = calls.apply(lambda row: row['Delta'] / float(row['Ask']), axis = 1)
    calls = calls.replace('-', '0')

    currentDate = datetime.today().strftime ('%Y-%m-%d')
    calls['Break Even Debit'] = calls.apply(lambda row :  float(row['Ask']) + float(row['Strike']), axis= 1)
    calls['Days to Exp'] = calls.apply(lambda row : days_between(currentDate, row['Expiration']), axis = 1)
    calls['Cost per Day'] = calls.apply(lambda row: float(row['Ask']) * 100 / float(row['Days to Exp']), axis = 1)
    calls = calls[calls['Delta'] > minDelta]
    if reverse == 0:
        sortedCalls = calls[calls['Expiration'] >= minDate].sort_values(by='Delta per Collateral', ascending = False)
    else:
        sortedCalls = calls[calls['Expiration'] >= minDate].sort_values(by='Delta per Collateral')
    print("Delta/Collateral of buying the stock is ", round(1/price, 6))
    return(sortedCalls[:50])

In [ ]:
calls_delta_per_collateral(ticker='TSLA', minDate = '2022-01-01', minDelta = 0.5, reverse = 0)

In [ ]:
def how_do_i_get_rich_fast():
    print("Get very lucky")

In [ ]:
how_do_i_get_rich_fast()

# To Do:

1. implement PMCC scanner
2. figure out how to do a PMCC scanner

## Poor Man's Covered Puts

In [ ]:
def puts_delta_per_collateral(ticker, minDate = '2020-12-31', maxDelta = 0, reverse = 0):

    with contextlib.redirect_stdout(None):
        expirationDates = list(set(robin_stocks.options.find_tradable_options_for_stock(ticker, optionType = 'put', info = 'expiration_date')))

    puts = pd.DataFrame (columns = ['Strike','Ask','Bid','Delta', 'Theta', 'Price', 'Expiration'])

    for j in range(0, len(expirationDates)):

        try:
            with contextlib.redirect_stdout(None):
                #option_info = options.get_calls(ticker, date = expirationDates[j])    

                pre_option_info = robin_stocks.find_options_for_stock_by_expiration(ticker, expirationDate=expirationDates[j], optionType='put')
            data = [ (item['strike_price'], item['ask_price'], item['bid_price'],  to_float_with_none_check(item['delta']), to_float_with_none_check(item['theta'])) for item in pre_option_info]
            option_info  = pd.DataFrame(data, columns = ['Strike', 'Ask', 'Bid', 'Delta', 'Theta'])
        except:
            print("EXCEPTION: ", expirationDates[j], " - ", ticker)
            continue

        tmpDF = option_info.reindex(columns=['Strike', 'Ask', 'Bid', 'Delta', 'Theta'])
        tmpDF['Expiration'] = expirationDates[j]
        puts = puts.append(tmpDF, sort = True)
    try:
        price = round(si.get_live_price(ticker),2)
    except:
        print("Issue getting price for", ticker)
        price = -1    
    puts['Price'] = price
    puts['Ticker'] = ticker
    puts['Delta per Collateral'] = puts.apply(lambda row: row['Delta'] / float(row['Ask']), axis = 1)
    puts = puts.replace('-', '0')

    currentDate = datetime.today().strftime ('%Y-%m-%d')
    puts['Break Even Debit'] = puts.apply(lambda row :  float(row['Strike']) - float(row['Ask']), axis= 1)
    puts['Days to Exp'] = puts.apply(lambda row : days_between(currentDate, row['Expiration']), axis = 1)
    puts['Cost per Day'] = puts.apply(lambda row: float(row['Ask']) * 100 / float(row['Days to Exp']), axis = 1)
    puts = puts[puts['Delta'] <= maxDelta]
    if reverse == 0:
        sortedPuts = puts[puts['Expiration'] >= minDate].sort_values(by='Delta per Collateral', ascending = True)
    else:
        sortedPuts = puts[puts['Expiration'] >= minDate].sort_values(by='Delta per Collateral')
    print("Delta/Collateral of buying the stock is ", round(1/price, 6))
    return(sortedPuts[:50])

In [ ]:
puts_delta_per_collateral(ticker='RIOT', minDate = '2021-12-31', maxDelta = -0.15, reverse = 0)

In [ ]:
def find_put_LEAPS_by_break_even_debit(ticker, minDate='2021-01-01', maxDelta=-0.15):


    with contextlib.redirect_stdout(None):
        expirationDates = list(set(robin_stocks.options.find_tradable_options_for_stock(ticker, optionType = 'put', info = 'expiration_date')))

    puts = pd.DataFrame (columns = ['Ticker', 'Strike','Ask','Bid', 'Price', 'Expiration'])

    for j in range(0, len(expirationDates)):
        try:
            with contextlib.redirect_stdout(None):
                pre_option_info = robin_stocks.find_options_for_stock_by_expiration(ticker, expirationDate=expirationDates[j], optionType='put')
            data = [ (item['strike_price'], item['ask_price'], item['bid_price'],  to_float_with_none_check(item['delta'])) for item in pre_option_info]
            option_info  = pd.DataFrame(data, columns = ['Strike', 'Ask', 'Bid', 'Delta'])
        except:
            print(expirationDates[j], "issue")
            continue
        tmpDF = option_info.reindex(columns=['Strike', 'Ask', 'Bid', 'Delta'])
        tmpDF['Expiration'] = expirationDates[j]

        puts = puts.append(tmpDF, sort = True)
    try:
        price = round(si.get_live_price(ticker),2)
    except:
        print("Issue getting price for", ticker)
        price = -1    
    puts['Price'] = price
    puts['Ticker'] = ticker
    puts = puts.replace('-', '0')

    currentDate = datetime.today().strftime ('%Y-%m-%d')
    #calls['Delta'] = calls.apply(lambda row :  float(row['Delta']), axis= 1)
    puts['Break Even Debit'] = puts.apply(lambda row :  float(row['Strike'])-float(row['Ask']), axis= 1)
    puts['Days to Exp'] = puts.apply(lambda row : days_between(currentDate, row['Expiration']), axis = 1)
    puts['Cost per Day'] = puts.apply(lambda row: float(row['Ask']) * 100 / float(row['Days to Exp']), axis = 1)
    puts = puts[puts['Delta'] < 0]
    sortedPuts = puts[puts['Expiration'] >= minDate].sort_values(by='Break Even Debit', ascending = False)
    sortedPuts = sortedPuts[sortedPuts['Delta'] <= maxDelta]
    #sortedPuts[:50]
    return(sortedPuts[:50])